# Python for Finance: Beta Weighting your Portfolio

<b> YouTube Tutorial </b> (Published: Jun 10, 2021): https://youtu.be/Zc67XB4voC4

In this tutorial we begin by using pandas_datareader to import financial stock data.
    https://pydata.github.io/pandas-datareader/index.html

In [ ]:
## This is required for pandas_datareader on google colab - then you need to restart runtime
!pip install --upgrade pandas_datareader

     |████████████████████████████████| 109 kB 5.2 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
from pandas_datareader import data as pdr

#### Step 1: Specify date range for analysis
Here we begin by creating start and end dates using pythons datetime module.

In [ ]:
start = dt.datetime(2021, 1, 1)
end = dt.datetime.now()
start, end

(datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2022, 2, 14, 5, 46, 28, 19089))

#### Step 2: Select the stocks/tickers you would like to analyse
For Australian stocks, yahoo tickers require '.AX' to be specified at the end of the ticker symbol.

For other tickers, use the search bar in yahoo finance to work out other ticker structures.
https://au.finance.yahoo.com/

In [ ]:
stockList = ['CBA', 'NAB', 'WBC', 'ANZ','WPL']
stocks = ['^AXJO'] + [i + '.AX' for i in stockList]
stocks

['^AXJO', 'CBA.AX', 'NAB.AX', 'WBC.AX', 'ANZ.AX', 'WPL.AX']

#### Step 3 call the Pandas_Datareader DataReader module:
Two ways of doing this:
1. pdr.DataReader(stocks, 'yahoo', start, end)
2. pdr.get_data_yahoo(stocks, start, end)

In [ ]:
df = pdr.get_data_yahoo(stocks, start, end)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

Symbols,^AXJO,CBA.AX,NAB.AX,WBC.AX,ANZ.AX,WPL.AX
Date,,,,,,
2021-01-04,-0.000344,-0.006228,-0.010522,-0.005619,-0.003043,-0.012211
2021-01-05,-0.011258,-0.003973,-0.012417,-0.007714,-0.008306,0.016968
2021-01-06,0.015752,0.020063,0.027725,0.031507,0.037057,0.049242
2021-01-07,0.006815,0.012338,0.013365,0.014403,0.008425,0.018715
2021-01-10,-0.009023,-0.005504,-0.006014,0.000493,0.000419,0.025862


#### Step 4a: Directly calculate beta:

$\frac{covariance(Market, Stock)}{variance(Market)}$

In [ ]:
def calc_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns are indexed by ind
            s = np_array[:,ind]
            # Calculate covariance matrix between stock and market
            covariance = np.cov(s,m)
            beta.append( covariance[0,1]/covariance[1,1] )
    return pd.Series(beta, df.columns[1:], name='Beta')

In [ ]:
calc_beta(log_returns)

Symbols
CBA.AX    1.154583
NAB.AX    0.951789
WBC.AX    0.885018
ANZ.AX    0.905474
WPL.AX    1.150844
Name: Beta, dtype: float64

#### Step 4b: Use linear regression to get coefficient of market and stocks returns

In [ ]:
def regression_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:,ind] # stock returns are column one from numpy array
            beta.append( stats.linregress(m,s)[0] )
    return pd.Series(beta, df.columns[1:], name='Beta')

In [ ]:
regression_beta(log_returns)

Symbols
CBA.AX    1.154583
NAB.AX    0.951789
WBC.AX    0.885018
ANZ.AX    0.905474
WPL.AX    1.150844
Name: Beta, dtype: float64

#### Step 4c: Use Matrix Algebra to complete linear regression in one line

For linear regression on a model of the form y=Xβ, where X is a matrix with full column rank, the least squares solution,

$\hat{\beta} = arg \min ||X\beta−y||_2 $


$\hat{\beta} = (X^T X)^{−1}X^Ty $

https://stats.stackexchange.com/questions/23128/solving-for-regression-parameters-in-closed-form-vs-gradient-descent/23132#23132

In [ ]:
def matrix_beta(df):
    # Market index is the first column 0
    X = df.values[:, [0]]
    # add an additional column for the intercept (initalise as 1's)
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # Apply matrix algebra for linear regression model
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    return pd.Series(beta[1], df.columns[1:], name='Beta')

In [ ]:
beta = matrix_beta(log_returns)
beta

Symbols
CBA.AX    1.154583
NAB.AX    0.951789
WBC.AX    0.885018
ANZ.AX    0.905474
WPL.AX    1.150844
Name: Beta, dtype: float64

#### Step 5: Define your Portfolio and make DataFrame

Calculate Beta Weighted Portfolio

In [ ]:
units = np.array([100, 250, 300, 400, 200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = [unit*pr for unit, pr in zip(units, price)]
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

In [ ]:
Portfolio = pd.DataFrame({
    'Stock': stockList,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': units*price,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Weight,Beta,Weighted Beta
Symbols,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.04,100,10004.0,0.24,1.15,0.2760
NAB.AX,NAB,Long,S,30.43,30.43,250,7607.5,0.18,0.95,0.1710
WBC.AX,WBC,Long,S,23.88,23.88,300,7164.0,0.17,0.89,0.1513
ANZ.AX,ANZ,Long,S,28.20,28.20,400,11280.0,0.27,0.91,0.2457
WPL.AX,WPL,Long,S,27.43,27.43,200,5486.0,0.13,1.15,0.1495


#### Step 6: What if we have options, let's consider things in terms of Delta

In [ ]:
Portfolio = Portfolio.drop(['Weight', 'Weighted Beta'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.04,100,10004.0,1.15,100
NAB.AX,NAB,Long,S,30.43,30.43,250,7607.5,0.95,250
WBC.AX,WBC,Long,S,23.88,23.88,300,7164.0,0.89,300
ANZ.AX,ANZ,Long,S,28.20,28.20,400,11280.0,0.91,400
WPL.AX,WPL,Long,S,27.43,27.43,200,5486.0,1.15,200


#### Add Options to portfolio
This is Only an example

In [ ]:
Options = [{'option':'CBA0Z8', 'underlying':'CBA', 'price':3.950, 'units': 2, 'delta': 0.627, 'direction': 'Short', 'type': 'Call'},
           {'option':'WPLQB9', 'underlying':'WPL', 'price':1.325, 'units': 2, 'delta': -0.425 ,'direction': 'Long', 'type': 'Put'}]

In [ ]:
for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.AX', 'Price'],
                                    row['price'], row['units'], row['price']*row['units']*100, beta[row['underlying']+'.AX'],
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100,10004.0,1.15,100.0
NAB.AX,NAB,Long,S,30.43,30.430,250,7607.5,0.95,250.0
WBC.AX,WBC,Long,S,23.88,23.880,300,7164.0,0.89,300.0
ANZ.AX,ANZ,Long,S,28.20,28.200,400,11280.0,0.91,400.0
WPL.AX,WPL,Long,S,27.43,27.430,200,5486.0,1.15,200.0
CBA0Z8,CBA,Short,Call,100.04,3.950,2,790.0,1.15,-125.4
WPLQB9,WPL,Long,Put,27.43,1.325,2,265.0,1.15,-85.0


#### Step 7: Weight the Delta's using Beta

In [ ]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']/ASXprices[0]) * Portfolio['Delta'],2)
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 0.01,2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Symbols,,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100,10004.0,1.15,100.0,1.59,115.05
NAB.AX,NAB,Long,S,30.43,30.430,250,7607.5,0.95,250.0,1.00,72.27
WBC.AX,WBC,Long,S,23.88,23.880,300,7164.0,0.89,300.0,0.88,63.76
ANZ.AX,ANZ,Long,S,28.20,28.200,400,11280.0,0.91,400.0,1.42,102.65
WPL.AX,WPL,Long,S,27.43,27.430,200,5486.0,1.15,200.0,0.87,63.09
CBA0Z8,CBA,Short,Call,100.04,3.950,2,790.0,1.15,-125.4,-1.99,-144.27
WPLQB9,WPL,Long,Put,27.43,1.325,2,265.0,1.15,-85.0,-0.37,-26.81


#### Step 8: Total the Delta's to get Portfolio Overview

In [ ]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] \
= Portfolio[['Value','ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Symbols,,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100.0,10004.0,1.15,100.0,1.59,115.05
NAB.AX,NAB,Long,S,30.43,30.430,250.0,7607.5,0.95,250.0,1.00,72.27
WBC.AX,WBC,Long,S,23.88,23.880,300.0,7164.0,0.89,300.0,0.88,63.76
ANZ.AX,ANZ,Long,S,28.20,28.200,400.0,11280.0,0.91,400.0,1.42,102.65
WPL.AX,WPL,Long,S,27.43,27.430,200.0,5486.0,1.15,200.0,0.87,63.09
CBA0Z8,CBA,Short,Call,100.04,3.950,2.0,790.0,1.15,-125.4,-1.99,-144.27
WPLQB9,WPL,Long,Put,27.43,1.325,2.0,265.0,1.15,-85.0,-0.37,-26.81
Total,NaN,NaN,NaN,NaN,NaN,NaN,42596.5,NaN,NaN,3.40,245.74
